In [ ]:
# Requires audio_biggerthan2sec/

In [ ]:
from IPython.display import Audio
import soundfile as sf
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
import os
import librosa
import librosa.display
from tqdm import tqdm_notebook
from pathlib import Path
import shutil
import numpy as np
import scipy
from random import shuffle

In [ ]:
threshold = 256

In [ ]:
for split in ['train','dev','eval']:
    for label in ['real','fake']:
        Path(f'mel_spec/{split}/{label}').mkdir(parents=True, exist_ok = True)

In [ ]:
n_mels = 256
fmin = 20
fmax = 8192
hop_length = 128
n_fft = 1024
for split in ['train','dev','eval']:
    for label in ['real','fake']:
        files = [f for f in os.listdir(f'audio_biggerthan2sec/{split}/{label}') if 'flac' in f]
        for file in tqdm_notebook(files, desc = f'{split}_{label}'):
            clip, sample_rate = librosa.load(f'audio_biggerthan2sec/{split}/{label}/{file}', sr=None)
            mel_spec = librosa.feature.melspectrogram(clip, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, sr=sample_rate, power=1.0, fmin=fmin, fmax=fmax)
            if mel_spec.shape[0] > mel_spec.shape[1]:
                continue
            mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)
            filename = file.split('.')[0]
            plt.imsave(f'mel_spec/{split}/{label}/{filename}.jpg',mel_spec_db[::-1,:],cmap='gray')

In [ ]:
og_dataset = 'mel_spec'
new_dataset_name = f'{og_dataset}_{threshold}_crop_first'

for split in ['train','dev','eval']:
    for label in ['real','fake']:
        Path(f'{new_dataset_name}/{split}/{label}').mkdir(parents=True, exist_ok = True)

for split in ['train','dev','eval']:
    for label in ['real','fake']:
        files = os.listdir(f'{og_dataset}/{split}/{label}')
        for file in tqdm_notebook(files, desc = f'{split}_{label}'):
            img = plt.imread(f'{og_dataset}/{split}/{label}/{file}')
            if img.shape[1] >= threshold:
                plt.imsave(f'{new_dataset_name}/{split}/{label}/{file}',img[:,:threshold],cmap = 'gray')

In [ ]:
for split in ['train','dev','eval']:
    for label in ['real','fake']:
        Path(f'amp_plot/{split}/{label}').mkdir(parents=True, exist_ok = True)

In [ ]:
scale = 4.72519513245
for split in ['train','dev','eval']:
    for label in ['real','fake']:
        files = [f for f in os.listdir(f'audio_biggerthan2sec/{split}/{label}') if 'flac' in f]
        count = 0
        for file in tqdm_notebook(files, desc = f'{split}_{label}'):
            clip, sample_rate = librosa.load(f'audio_biggerthan2sec/{split}/{label}/{file}', sr=None)
            if len(clip)/sample_rate >= 2:
                plt.figure(figsize=((len(clip)/sample_rate)*scale/2,scale))
                plt.plot(clip,color='k')
                plt.axis('off')
                plt.margins(0,0)
                plt.savefig(f'amp_plot/{split}/{label}/{file}.jpg', cmap='gray', bbox_inches='tight', pad_inches = 0)
                if count % 20 == 0:
                    plt.close('all')
            count += 1

In [ ]:
og_dataset = 'amp_plot'
new_dataset_name = f'{og_dataset}_{threshold}_crop_first_negative'

for split in ['train','dev','eval']:
    for label in ['real','fake']:
        Path(f'{new_dataset_name}/{split}/{label}').mkdir(parents=True, exist_ok = True)

for split in ['train','dev','eval']:
    for label in ['real','fake']:
        files = os.listdir(f'{og_dataset}/{split}/{label}')
        for file in tqdm_notebook(files, desc = f'{split}_{label}'):
            img = plt.imread(f'{og_dataset}/{split}/{label}/{file}')
            if img.shape[1] >= threshold:
                plt.imsave(f'{new_dataset_name}/{split}/{label}/{file}',255-img[:,:threshold],cmap = 'gray')